# 15 diffusion models keras
**Location: TensorVerseHub/notebooks/05_generative_models/15_diffusion_models_keras.ipynb**

TODO: Implement comprehensive TensorFlow + tf.keras learning content.

## Learning Objectives
- TODO: Define specific learning objectives
- TODO: List key TensorFlow concepts covered
- TODO: Outline tf.keras integration points

In [ ]:
import tensorflow as tf
import numpy as np
print(f"TensorFlow version: {tf.__version__}")
# TODO: Add comprehensive implementation

# Diffusion Models with tf.keras

**File Location:** `notebooks/05_generative_models/15_diffusion_models_keras.ipynb`

Master diffusion models including DDPM, DDIM, and advanced sampling techniques using tf.keras. Implement state-of-the-art generative models that have revolutionized image synthesis through iterative denoising processes.

## Learning Objectives
- Understand diffusion model theory and forward/reverse processes
- Implement DDPM (Denoising Diffusion Probabilistic Models) with tf.keras
- Build U-Net architectures with attention mechanisms for denoising
- Master DDIM (Denoising Diffusion Implicit Models) for faster sampling
- Apply classifier-free guidance for controllable generation
- Implement advanced sampling strategies and noise schedules

---

## 1. Diffusion Model Fundamentals

```python
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers
import math
import time
import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
tf.random.set_seed(42)
np.random.seed(42)

# Core diffusion model components
class DiffusionScheduler:
    """Manages noise schedules for diffusion models"""
    
    def __init__(self, timesteps=1000, beta_start=0.0001, beta_end=0.02, schedule_type='linear'):
        self.timesteps = timesteps
        self.schedule_type = schedule_type
        
        # Create noise schedule
        if schedule_type == 'linear':
            self.betas = np.linspace(beta_start, beta_end, timesteps)
        elif schedule_type == 'cosine':
            self.betas = self.cosine_beta_schedule(timesteps)
        else:
            raise ValueError(f"Unknown schedule type: {schedule_type}")
        
        # Convert to tensors
        self.betas = tf.constant(self.betas, dtype=tf.float32)
        
        # Pre-compute useful quantities
        self.alphas = 1.0 - self.betas
        self.alphas_cumprod = tf.math.cumprod(self.alphas, axis=0)
        self.alphas_cumprod_prev = tf.concat([tf.ones(1), self.alphas_cumprod[:-1]], axis=0)
        
        # For reverse process
        self.sqrt_alphas_cumprod = tf.sqrt(self.alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = tf.sqrt(1.0 - self.alphas_cumprod)
        
        # For sampling
        self.posterior_variance = self.betas * (1.0 - self.alphas_cumprod_prev) / (1.0 - self.alphas_cumprod)
        self.posterior_log_variance_clipped = tf.math.log(tf.maximum(self.posterior_variance, 1e-20))
        
        print(f"Diffusion scheduler initialized with {timesteps} timesteps")
        print(f"Beta range: [{beta_start:.6f}, {beta_end:.6f}]")
        print(f"Schedule type: {schedule_type}")
    
    def cosine_beta_schedule(self, timesteps, s=0.008):
        """Cosine noise schedule as proposed in Improved DDPM"""
        steps = timesteps + 1
        x = np.linspace(0, timesteps, steps)
        alphas_cumprod = np.cos(((x / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
        alphas_cumprod = alphas_cumprod / alphas_cumprod[0]
        betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
        return np.clip(betas, 0, 0.999)
    
    def add_noise(self, x_0, t, noise=None):
        """Forward diffusion process: add noise to clean images"""
        if noise is None:
            noise = tf.random.normal(tf.shape(x_0))
        
        # Gather values for timestep t
        sqrt_alphas_cumprod_t = tf.gather(self.sqrt_alphas_cumprod, t)
        sqrt_one_minus_alphas_cumprod_t = tf.gather(self.sqrt_one_minus_alphas_cumprod, t)
        
        # Reshape for broadcasting
        sqrt_alphas_cumprod_t = tf.reshape(sqrt_alphas_cumprod_t, [-1, 1, 1, 1])
        sqrt_one_minus_alphas_cumprod_t = tf.reshape(sqrt_one_minus_alphas_cumprod_t, [-1, 1, 1, 1])
        
        # q(x_t | x_0)
        x_t = sqrt_alphas_cumprod_t * x_0 + sqrt_one_minus_alphas_cumprod_t * noise
        return x_t, noise

# U-Net architecture for denoising
class UNetBlock(layers.Layer):
    """U-Net residual block with time embedding"""
    
    def __init__(self, channels, time_emb_dim=None, dropout_rate=0.1, **kwargs):
        super().__init__(**kwargs)
        self.channels = channels
        self.time_emb_dim = time_emb_dim
        
        # Group normalization
        self.norm1 = layers.GroupNormalization(groups=min(32, channels))
        self.norm2 = layers.GroupNormalization(groups=min(32, channels))
        
        # Convolutions
        self.conv1 = layers.Conv2D(channels, 3, padding='same')
        self.conv2 = layers.Conv2D(channels, 3, padding='same')
        
        # Time embedding projection
        if time_emb_dim:
            self.time_mlp = layers.Dense(channels)
        
        # Dropout
        self.dropout = layers.Dropout(dropout_rate)
    
    def call(self, x, time_emb=None, training=None):
        # First conv block
        h = self.norm1(x)
        h = tf.nn.swish(h)
        h = self.conv1(h)
        
        # Add time embedding
        if time_emb is not None and self.time_emb_dim:
            time_proj = self.time_mlp(tf.nn.swish(time_emb))
            h += tf.expand_dims(tf.expand_dims(time_proj, 1), 1)
        
        # Second conv block
        h = self.norm2(h)
        h = tf.nn.swish(h)
        h = self.dropout(h, training=training)
        h = self.conv2(h)
        
        # Residual connection
        if tf.shape(x)[-1] != self.channels:
            x = layers.Conv2D(self.channels, 1)(x)
        
        return h + x

class UNetDiffusionModel(tf.keras.Model):
    """U-Net model for diffusion denoising"""
    
    def __init__(self, img_shape=(32, 32, 3), time_emb_dim=128, base_channels=64, **kwargs):
        super().__init__(**kwargs)
        
        self.img_shape = img_shape
        self.time_emb_dim = time_emb_dim
        
        # Time embedding
        self.time_mlp = tf.keras.Sequential([
            layers.Dense(time_emb_dim, activation='swish'),
            layers.Dense(time_emb_dim)
        ])
        
        # Encoder
        self.init_conv = layers.Conv2D(base_channels, 3, padding='same')
        
        self.encoder_blocks = [
            UNetBlock(base_channels, time_emb_dim),
            UNetBlock(base_channels * 2, time_emb_dim),
            UNetBlock(base_channels * 4, time_emb_dim),
        ]
        
        self.downsample_blocks = [
            layers.Conv2D(base_channels * 2, 3, strides=2, padding='same'),
            layers.Conv2D(base_channels * 4, 3, strides=2, padding='same'),
            None
        ]
        
        # Middle
        self.mid_block = UNetBlock(base_channels * 4, time_emb_dim)
        
        # Decoder
        self.upsample_blocks = [
            layers.Conv2DTranspose(base_channels * 2, 3, strides=2, padding='same'),
            layers.Conv2DTranspose(base_channels, 3, strides=2, padding='same'),
            None
        ]
        
        self.decoder_blocks = [
            UNetBlock(base_channels * 4, time_emb_dim),
            UNetBlock(base_channels * 2, time_emb_dim),
            UNetBlock(base_channels, time_emb_dim),
        ]
        
        # Final output
        self.final_norm = layers.GroupNormalization(groups=8)
        self.final_conv = layers.Conv2D(img_shape[-1], 3, padding='same')
    
    def time_embedding(self, timesteps):
        """Sinusoidal time embeddings"""
        half_dim = self.time_emb_dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
        emb = tf.cast(timesteps, tf.float32)[:, None] * emb[None, :]
        emb = tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
        return self.time_mlp(emb)
    
    def call(self, x, timesteps, training=None):
        # Time embedding
        time_emb = self.time_embedding(timesteps)
        
        # Encoder
        h = self.init_conv(x)
        skip_connections = []
        
        for encoder_block, downsample in zip(self.encoder_blocks, self.downsample_blocks):
            h = encoder_block(h, time_emb, training=training)
            skip_connections.append(h)
            if downsample:
                h = downsample(h)
        
        # Middle
        h = self.mid_block(h, time_emb, training=training)
        
        # Decoder
        for upsample, decoder_block, skip in zip(self.upsample_blocks, self.decoder_blocks, reversed(skip_connections)):
            if upsample:
                h = upsample(h)
            h = tf.concat([h, skip], axis=-1)
            h = decoder_block(h, time_emb, training=training)
        
        # Final output
        h = self.final_norm(h)
        h = tf.nn.swish(h)
        return self.final_conv(h)

# Test basic components
print("=== Testing Diffusion Components ===")

# Initialize scheduler
scheduler = DiffusionScheduler(timesteps=1000, schedule_type='cosine')

# Load sample data
(x_train, _), (x_test, _) = tf.keras.datasets.cifar10.load_data()
x_train = x_train.astype('float32') / 255.0
x_train = x_train * 2.0 - 1.0  # Scale to [-1, 1]

print(f"CIFAR-10 data: {x_train.shape}, range: [{x_train.min():.2f}, {x_train.max():.2f}]")

# Test forward diffusion
batch_size = 4
sample_images = x_train[:batch_size]
timesteps = tf.constant([100, 300, 600, 900])

# Add noise at different timesteps
noisy_images = []
for i, t in enumerate(timesteps):
    x_t, noise = scheduler.add_noise(sample_images[i:i+1], tf.constant([t]))
    noisy_images.append(x_t[0])

# Visualize forward process
plt.figure(figsize=(16, 8))
for i in range(4):
    # Original
    plt.subplot(2, 4, i + 1)
    plt.imshow((sample_images[i] + 1) / 2)
    plt.title(f'Original')
    plt.axis('off')
    
    # Noisy
    plt.subplot(2, 4, i + 5)
    plt.imshow((noisy_images[i] + 1) / 2)
    plt.title(f't={timesteps[i]}')
    plt.axis('off')

plt.suptitle('Forward Diffusion Process')
plt.tight_layout()
plt.show()

# Test U-Net
print("\n=== Testing U-Net Architecture ===")
unet = UNetDiffusionModel(img_shape=(32, 32, 3), time_emb_dim=128, base_channels=64)

# Build model
sample_batch = sample_images
sample_timesteps = tf.constant([100, 200, 300, 400])
predicted_noise = unet(sample_batch, sample_timesteps)

print(f"U-Net Model:")
print(f"Input shape: {sample_batch.shape}")
print(f"Output shape: {predicted_noise.shape}")
print(f"Parameters: {unet.count_params():,}")
```

## 2. DDPM Implementation

```python
# Denoising Diffusion Probabilistic Model (DDPM)
class DDPM:
    """Complete DDPM implementation with training and sampling"""
    
    def __init__(self, unet_model, scheduler, img_shape=(32, 32, 3)):
        self.unet = unet_model
        self.scheduler = scheduler
        self.img_shape = img_shape
        
        # Optimizer
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
        
        # Loss tracking
        self.loss_tracker = tf.keras.metrics.Mean(name="loss")
    
    def train_step(self, batch):
        """Single training step for DDPM"""
        
        batch_size = tf.shape(batch)[0]
        
        # Sample random timesteps
        timesteps = tf.random.uniform([batch_size], 0, self.scheduler.timesteps, dtype=tf.int32)
        
        # Sample noise
        noise = tf.random.normal(tf.shape(batch))
        
        # Forward diffusion: add noise to images
        noisy_images, _ = self.scheduler.add_noise(batch, timesteps, noise)
        
        with tf.GradientTape() as tape:
            # Predict noise using U-Net
            predicted_noise = self.unet(noisy_images, timesteps, training=True)
            
            # MSE loss between predicted and actual noise
            loss = tf.reduce_mean(tf.square(noise - predicted_noise))
        
        # Compute gradients and update weights
        gradients = tape.gradient(loss, self.unet.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.unet.trainable_variables))
        
        # Update metrics
        self.loss_tracker.update_state(loss)
        
        return {"loss": self.loss_tracker.result()}
    
    @tf.function
    def train_step_compiled(self, batch):
        """Compiled version of training step for faster execution"""
        return self.train_step(batch)
    
    def sample(self, num_samples=4, num_steps=None, eta=0.0):
        """DDPM sampling (reverse diffusion)"""
        
        if num_steps is None:
            num_steps = self.scheduler.timesteps
        
        # Start from pure noise
        shape = [num_samples] + list(self.img_shape)
        img = tf.random.normal(shape)
        
        # Reverse diffusion
        timesteps = tf.range(self.scheduler.timesteps - 1, -1, -1)
        
        for i, t in enumerate(timesteps):
            if i % (num_steps // 10) == 0:
                print(f"Sampling step {i}/{num_steps}")
            
            # Current timestep
            t_batch = tf.fill([num_samples], t)
            
            # Predict noise
            predicted_noise = self.unet(img, t_batch, training=False)
            
            # Compute previous sample
            img = self.ddpm_step(img, predicted_noise, t)
        
        return img
    
    def ddpm_step(self, x_t, predicted_noise, t):
        """Single DDPM denoising step"""
        
        # Get scheduler parameters for timestep t
        alpha_t = tf.gather(self.scheduler.alphas, t)
        alpha_cumprod_t = tf.gather(self.scheduler.alphas_cumprod, t)
        alpha_cumprod_prev = tf.gather(self.scheduler.alphas_cumprod_prev, t)
        beta_t = tf.gather(self.scheduler.betas, t)
        
        # Reshape for broadcasting
        alpha_t = tf.reshape(alpha_t, [-1, 1, 1, 1])
        alpha_cumprod_t = tf.reshape(alpha_cumprod_t, [-1, 1, 1, 1])
        alpha_cumprod_prev = tf.reshape(alpha_cumprod_prev, [-1, 1, 1, 1])
        beta_t = tf.reshape(beta_t, [-1, 1, 1, 1])
        
        # Compute coefficients
        pred_original_sample_coeff = 1 / tf.sqrt(alpha_cumprod_t)
        current_sample_coeff = tf.sqrt(alpha_cumprod_prev) * beta_t / (1 - alpha_cumprod_t)
        
        # Predict x_0
        pred_original_sample = pred_original_sample_coeff * (x_t - tf.sqrt(1 - alpha_cumprod_t) * predicted_noise)
        
        # Compute x_{t-1}
        pred_prev_sample = current_sample_coeff * pred_original_sample
        pred_prev_sample += tf.sqrt(alpha_t) * (1 - alpha_cumprod_prev) / (1 - alpha_cumprod_t) * x_t
        
        # Add noise if not the final step
        if t > 0:
            noise = tf.random.normal(tf.shape(x_t))
            variance = tf.gather(self.scheduler.posterior_variance, t)
            variance = tf.reshape(variance, [-1, 1, 1, 1])
            pred_prev_sample += tf.sqrt(variance) * noise
        
        return pred_prev_sample
    
    def fit(self, dataset, epochs=100, steps_per_epoch=100):
        """Train DDPM model"""
        
        print(f"Training DDPM for {epochs} epochs...")
        
        history = {'loss': []}
        
        for epoch in range(epochs):
            epoch_start = time.time()
            
            # Reset metrics
            self.loss_tracker.reset_states()
            
            # Training loop
            for step, batch in enumerate(dataset.take(steps_per_epoch)):
                metrics = self.train_step_compiled(batch)
                
                if step % 50 == 0:
                    print(f"Epoch {epoch+1}/{epochs}, Step {step}: Loss = {metrics['loss']:.4f}")
            
            epoch_time = time.time() - epoch_start
            epoch_loss = self.loss_tracker.result()
            
            history['loss'].append(float(epoch_loss))
            
            print(f"Epoch {epoch+1}/{epochs} completed in {epoch_time:.2f}s - Loss: {epoch_loss:.4f}")
            
            # Sample images periodically
            if (epoch + 1) % 10 == 0:
                print("Generating samples...")
                samples = self.sample(num_samples=4, num_steps=100)
                self.visualize_samples(samples, epoch + 1)
        
        return history
    
    def visualize_samples(self, samples, epoch=None):
        """Visualize generated samples"""
        
        # Clip and rescale to [0, 1]
        samples = tf.clip_by_value((samples + 1) / 2, 0, 1)
        
        plt.figure(figsize=(12, 3))
        for i in range(min(4, samples.shape[0])):
            plt.subplot(1, 4, i + 1)
            plt.imshow(samples[i])
            plt.axis('off')
            plt.title(f'Sample {i+1}')
        
        title = f'Generated Samples - Epoch {epoch}' if epoch else 'Generated Samples'
        plt.suptitle(title)
        plt.tight_layout()
        plt.show()

# Train DDPM
print("=== Training DDPM ===")

# Create dataset
train_dataset = tf.data.Dataset.from_tensor_slices(x_train[:5000])
train_dataset = train_dataset.batch(16).prefetch(tf.data.AUTOTUNE)

# Initialize DDPM
ddpm = DDPM(unet, scheduler, img_shape=(32, 32, 3))

print("DDPM initialized. Starting training...")
print("Note: This is a demo with reduced epochs and data for faster execution")

# Train for few epochs (demo)
history = ddpm.fit(train_dataset, epochs=5, steps_per_epoch=20)

# Plot training loss
plt.figure(figsize=(10, 6))
plt.plot(history['loss'], marker='o')
plt.title('DDPM Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True, alpha=0.3)
plt.show()

# Generate final samples
print("\nGenerating final samples...")
final_samples = ddpm.sample(num_samples=8, num_steps=50)
ddpm.visualize_samples(final_samples)
```

## 3. DDIM and Fast Sampling

```python
# Denoising Diffusion Implicit Models (DDIM)
class DDIM:
    """DDIM implementation for faster sampling"""
    
    def __init__(self, unet_model, scheduler):
        self.unet = unet_model
        self.scheduler = scheduler
    
    def sample(self, num_samples=4, num_inference_steps=50, eta=0.0):
        """DDIM sampling with fewer steps"""
        
        # Create timestep schedule
        timesteps = self.get_timesteps(num_inference_steps)
        
        # Start from noise
        shape = [num_samples] + list(self.scheduler.img_shape if hasattr(self.scheduler, 'img_shape') else [32, 32, 3])
        image = tf.random.normal(shape)
        
        print(f"DDIM sampling with {num_inference_steps} steps...")
        
        for i, t in enumerate(timesteps):
            if i % 10 == 0:
                print(f"Step {i}/{num_inference_steps}")
            
            # Expand timestep to batch dimension
            t_batch = tf.fill([num_samples], t)
            
            # Predict noise
            noise_pred = self.unet(image, t_batch, training=False)
            
            # DDIM step
            image = self.ddim_step(image, noise_pred, t, 
                                 timesteps[i+1] if i < len(timesteps) - 1 else 0, 
                                 eta)
        
        return image
    
    def get_timesteps(self, num_inference_steps):
        """Create timestep schedule for DDIM"""
        
        # Linear spacing
        step_ratio = self.scheduler.timesteps // num_inference_steps
        timesteps = list(range(0, self.scheduler.timesteps, step_ratio))
        timesteps = timesteps[:num_inference_steps]
        
        return list(reversed(timesteps))
    
    def ddim_step(self, x_t, predicted_noise, t, t_prev, eta=0.0):
        """Single DDIM step"""
        
        # Get alpha values
        alpha_cumprod_t = tf.gather(self.scheduler.alphas_cumprod, t)
        alpha_cumprod_t_prev = tf.gather(self.scheduler.alphas_cumprod, t_prev) if t_prev >= 0 else tf.ones_like(alpha_cumprod_t)
        
        # Reshape for broadcasting
        alpha_cumprod_t = tf.reshape(alpha_cumprod_t, [-1, 1, 1, 1])
        alpha_cumprod_t_prev = tf.reshape(alpha_cumprod_t_prev, [-1, 1, 1, 1])
        
        # Predict x_0
        pred_original_sample = (x_t - tf.sqrt(1 - alpha_cumprod_t) * predicted_noise) / tf.sqrt(alpha_cumprod_t)
        
        # Compute coefficients
        pred_original_sample_coeff = tf.sqrt(alpha_cumprod_t_prev)
        current_sample_coeff = tf.sqrt(1 - alpha_cumprod_t_prev - eta**2 * (1 - alpha_cumprod_t_prev) * (1 - alpha_cumprod_t) / (1 - alpha_cumprod_t_prev))
        
        # Compute x_{t-1}
        pred_prev_sample = (pred_original_sample_coeff * pred_original_sample + 
                           current_sample_coeff * predicted_noise)
        
        # Add stochastic component if eta > 0
        if eta > 0 and t_prev > 0:
            noise = tf.random.normal(tf.shape(x_t))
            variance = eta**2 * (1 - alpha_cumprod_t_prev) * (1 - alpha_cumprod_t) / (1 - alpha_cumprod_t)
            variance = tf.reshape(variance, [-1, 1, 1, 1])
            pred_prev_sample += tf.sqrt(variance) * noise
        
        return pred_prev_sample

# Classifier-Free Guidance
class ClassifierFreeGuidance:
    """Classifier-free guidance for controllable generation"""
    
    def __init__(self, unet_model, scheduler, num_classes=10):
        self.unet = unet_model
        self.scheduler = scheduler
        self.num_classes = num_classes
        
        # Modify U-Net to accept class conditioning
        self.build_conditional_unet()
    
    def build_conditional_unet(self):
        """Build class-conditional U-Net"""
        
        # Add class embedding to existing U-Net
        self.class_embedding = layers.Embedding(self.num_classes + 1, 128)  # +1 for null class
        
        # Modified forward pass that combines time and class embeddings
        self.original_call = self.unet.call
        
        def conditional_call(x, timesteps, class_labels=None, training=None):
            # Time embedding
            time_emb = self.unet.time_embedding(timesteps)
            
            # Class embedding
            if class_labels is not None:
                class_emb = self.class_embedding(class_labels)
                # Combine time and class embeddings
                combined_emb = time_emb + class_emb
            else:
                combined_emb = time_emb
            
            # Use combined embedding in place of time embedding
            return self.forward_with_embedding(x, combined_emb, training)
        
        self.unet.conditional_call = conditional_call
    
    def forward_with_embedding(self, x, embedding, training):
        """Forward pass with combined embedding"""
        
        # This is a simplified version - in practice, you'd need to modify
        # all the U-Net blocks to accept the combined embedding
        
        # For demo, we'll use the original call but with modified time embedding
        original_time_mlp = self.unet.time_mlp
        
        # Temporarily replace time MLP output
        def mock_time_embedding(timesteps):
            return embedding
        
        self.unet.time_embedding = mock_time_embedding
        
        result = self.original_call(x, tf.zeros(tf.shape(x)[0], dtype=tf.int32), training)
        
        # Restore original method
        self.unet.time_embedding = lambda timesteps: self.unet.time_mlp(self.get_time_embedding(timesteps))
        
        return result
    
    def get_time_embedding(self, timesteps):
        """Original time embedding computation"""
        half_dim = self.unet.time_emb_dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = tf.exp(tf.range(half_dim, dtype=tf.float32) * -emb)
        emb = tf.cast(timesteps, tf.float32)[:, None] * emb[None, :]
        return tf.concat([tf.sin(emb), tf.cos(emb)], axis=-1)
    
    def sample_with_guidance(self, class_labels, guidance_scale=7.5, num_samples=4, num_steps=50):
        """Sample with classifier-free guidance"""
        
        batch_size = len(class_labels)
        
        # Create unconditional labels (null class = num_classes)
        uncond_labels = tf.fill([batch_size], self.num_classes)
        
        # Combine conditional and unconditional
        combined_labels = tf.concat([class_labels, uncond_labels], axis=0)
        
        # Start from noise
        shape = [batch_size] + [32, 32, 3]
        x_t = tf.random.normal(shape)
        
        # Timesteps
        timesteps = list(range(self.scheduler.timesteps - 1, -1, -num_steps))
        
        for i, t in enumerate(timesteps):
            if i % 10 == 0:
                print(f"Guidance sampling step {i}/{len(timesteps)}")
            
            # Duplicate input for conditional and unconditional
            x_input = tf.concat([x_t, x_t], axis=0)
            t_input = tf.fill([batch_size * 2], t)
            
            # Predict noise
            noise_pred = self.unet.conditional_call(x_input, t_input, combined_labels, training=False)
            
            # Split predictions
            noise_pred_cond, noise_pred_uncond = tf.split(noise_pred, 2, axis=0)
            
            # Apply guidance
            noise_pred = noise_pred_uncond + guidance_scale * (noise_pred_cond - noise_pred_uncond)
            
            # DDIM step
            x_t = self.ddim_step(x_t, noise_pred, t, 
                                timesteps[i+1] if i < len(timesteps) - 1 else 0)
        
        return x_t
    
    def ddim_step(self, x_t, predicted_noise, t, t_prev):
        """Simplified DDIM step for guidance"""
        
        alpha_cumprod_t = tf.gather(self.scheduler.alphas_cumprod, t)
        alpha_cumprod_t_prev = tf.gather(self.scheduler.alphas_cumprod, t_prev) if t_prev >= 0 else 1.0
        
        alpha_cumprod_t = tf.reshape(alpha_cumprod_t, [-1, 1, 1, 1])
        alpha_cumprod_t_prev = tf.reshape(alpha_cumprod_t_prev, [-1, 1, 1, 1])
        
        # Predict x_0
        pred_x0 = (x_t - tf.sqrt(1 - alpha_cumprod_t) * predicted_noise) / tf.sqrt(alpha_cumprod_t)
        
        # Compute x_{t-1}
        pred_prev_sample = (tf.sqrt(alpha_cumprod_t_prev) * pred_x0 + 
                           tf.sqrt(1 - alpha_cumprod_t_prev) * predicted_noise)
        
        return pred_prev_sample

# Test DDIM and advanced sampling
print("=== Testing DDIM and Advanced Sampling ===")

# DDIM sampling
ddim_sampler = DDIM(unet, scheduler)

print("Testing DDIM sampling...")
ddim_samples = ddim_sampler.sample(num_samples=4, num_inference_steps=20, eta=0.0)

# Visualize DDIM samples
plt.figure(figsize=(12, 6))

# DDIM samples
for i in range(4):
    plt.subplot(2, 4, i + 1)
    sample_img = tf.clip_by_value((ddim_samples[i] + 1) / 2, 0, 1)
    plt.imshow(sample_img)
    plt.title(f'DDIM Sample {i+1}')
    plt.axis('off')

# Compare with DDPM samples (if available)
if 'final_samples' in locals():
    for i in range(4):
        plt.subplot(2, 4, i + 5)
        sample_img = tf.clip_by_value((final_samples[i] + 1) / 2, 0, 1)
        plt.imshow(sample_img)
        plt.title(f'DDPM Sample {i+1}')
        plt.axis('off')

plt.suptitle('DDIM vs DDPM Sampling Comparison')
plt.tight_layout()
plt.show()

# Test classifier-free guidance (simplified demo)
print("\nTesting Classifier-Free Guidance...")

cfg_sampler = ClassifierFreeGuidance(unet, scheduler, num_classes=10)

# Sample with different class conditions
class_labels = tf.constant([0, 1, 2, 3])  # CIFAR-10 classes
guided_samples = cfg_sampler.sample_with_guidance(
    class_labels, guidance_scale=2.0, num_samples=4, num_steps=20
)

# Visualize guided samples
plt.figure(figsize=(12, 3))
for i in range(4):
    plt.subplot(1, 4, i + 1)
    sample_img = tf.clip_by_value((guided_samples[i] + 1) / 2, 0, 1)
    plt.imshow(sample_img)
    plt.title(f'Class {class_labels[i]} Guided')
    plt.axis('off')

plt.suptitle('Classifier-Free Guidance Results')
plt.tight_layout()
plt.show()

# Sampling time comparison
print("\nSampling Speed Comparison:")

# Time DDPM sampling
start_time = time.time()
ddpm_samples = ddpm.sample(num_samples=1, num_steps=50)
ddpm_time = time.time() - start_time

# Time DDIM sampling
start_time = time.time()
ddim_samples = ddim_sampler.sample(num_samples=1, num_inference_steps=20)
ddim_time = time.time() - start_time

print(f"DDPM (50 steps): {ddpm_time:.2f} seconds")
print(f"DDIM (20 steps): {ddim_time:.2f} seconds")
print(f"DDIM is {ddmp_time / ddim_time:.1f}x faster")
```

## 4. Advanced Techniques and Analysis

```python
# Noise schedule analysis
def analyze_noise_schedules():
    """Compare different noise schedules"""
    
    timesteps = 1000
    
    # Linear schedule
    linear_scheduler = DiffusionScheduler(timesteps, schedule_type='linear')
    
    # Cosine schedule
    cosine_scheduler = DiffusionScheduler(timesteps, schedule_type='cosine')
    
    # Plot comparison
    plt.figure(figsize=(15, 10))
    
    # Beta schedules
    plt.subplot(2, 3, 1)
    plt.plot(linear_scheduler.betas.numpy(), label='Linear', alpha=0.8)
    plt.plot(cosine_scheduler.betas.numpy(), label='Cosine', alpha=0.8)
    plt.title('Beta Schedules')
    plt.xlabel('Timestep')
    plt.ylabel('Beta')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Alpha cumulative product
    plt.subplot(2, 3, 2)
    plt.plot(linear_scheduler.alphas_cumprod.numpy(), label='Linear', alpha=0.8)
    plt.plot(cosine_scheduler.alphas_cumprod.numpy(), label='Cosine', alpha=0.8)
    plt.title('Cumulative Alpha Product')
    plt.xlabel('Timestep')
    plt.ylabel('Alpha Cumulative Product')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Signal-to-noise ratio
    plt.subplot(2, 3, 3)
    snr_linear = linear_scheduler.alphas_cumprod / (1 - linear_scheduler.alphas_cumprod)
    snr_cosine = cosine_scheduler.alphas_cumprod / (1 - cosine_scheduler.alphas_cumprod)
    
    plt.plot(10 * tf.math.log(snr_linear) / tf.math.log(10.0), label='Linear', alpha=0.8)
    plt.plot(10 * tf.math.log(snr_cosine) / tf.math.log(10.0), label='Cosine', alpha=0.8)
    plt.title('Signal-to-Noise Ratio (dB)')
    plt.xlabel('Timestep')
    plt.ylabel('SNR (dB)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Noise levels visualization
    sample_image = x_train[0:1]
    timesteps_to_show = [0, 200, 400, 600, 800, 999]
    
    for i, (t, scheduler, name) in enumerate([(t, linear_scheduler, 'Linear') for t in [200, 600]] + 
                                             [(t, cosine_scheduler, 'Cosine') for t in [200, 600]]):
        noisy_image, _ = scheduler.add_noise(sample_image, tf.constant([t]))
        
        plt.subplot(2, 3, 4 + i)
        plt.imshow((noisy_image[0] + 1) / 2)
        plt.title(f'{name} Schedule at t={t}')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()

# Loss landscape analysis
def analyze_training_dynamics(ddpm_model, dataset, num_batches=50):
    """Analyze training dynamics and loss components"""
    
    losses = []
    timestep_losses = {t: [] for t in range(0, ddmp_model.scheduler.timesteps, 100)}
    
    for batch_idx, batch in enumerate(dataset.take(num_batches)):
        batch_size = tf.shape(batch)[0]
        
        # Analyze loss at different timesteps
        for t in range(0, ddpm_model.scheduler.timesteps, 100):
            timesteps = tf.fill([batch_size], t)
            noise = tf.random.normal(tf.shape(batch))
            
            noisy_images, _ = ddpm_model.scheduler.add_noise(batch, timesteps, noise)
            predicted_noise = ddpm_model.unet(noisy_images, timesteps, training=False)
            
            loss = tf.reduce_mean(tf.square(noise - predicted_noise))
            timestep_losses[t].append(loss.numpy())
        
        if batch_idx % 10 == 0:
            print(f"Analyzed batch {batch_idx}/{num_batches}")
    
    # Plot timestep-specific losses
    plt.figure(figsize=(12, 8))
    
    timesteps = list(timestep_losses.keys())
    mean_losses = [np.mean(timestep_losses[t]) for t in timesteps]
    std_losses = [np.std(timestep_losses[t]) for t in timesteps]
    
    plt.errorbar(timesteps, mean_losses, yerr=std_losses, capsize=5, alpha=0.8)
    plt.title('Loss vs Timestep Analysis')
    plt.xlabel('Timestep')
    plt.ylabel('MSE Loss')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    return timestep_losses

# Quality metrics for diffusion models
class DiffusionMetrics:
    """Evaluation metrics for diffusion models"""
    
    def __init__(self):
        pass
    
    def compute_fid_score(self, real_images, generated_images, batch_size=50):
        """Compute FID score (simplified version)"""
        
        def get_inception_features(images):
            # In practice, use pre-trained InceptionV3
            # This is a simplified version
            model = tf.keras.Sequential([
                layers.Rescaling(255.0),  # Scale back to [0, 255]
                layers.Lambda(lambda x: tf.image.resize(x, [299, 299])),
                tf.keras.applications.InceptionV3(include_top=False, pooling='avg', weights='imagenet')
            ])
            return model(images)
        
        # Get features
        real_features = get_inception_features(real_images[:batch_size])
        fake_features = get_inception_features(generated_images[:batch_size])
        
        # Calculate statistics
        mu_real = tf.reduce_mean(real_features, axis=0)
        mu_fake = tf.reduce_mean(fake_features, axis=0)
        
        sigma_real = tfp.stats.covariance(real_features)
        sigma_fake = tfp.stats.covariance(fake_features)
        
        # FID calculation (simplified)
        diff = mu_real - mu_fake
        fid = tf.reduce_sum(diff ** 2)
        
        return fid.numpy()
    
    def compute_lpips_score(self, images1, images2):
        """Compute LPIPS (perceptual similarity)"""
        
        # Simplified version using VGG features
        vgg = tf.keras.applications.VGG16(include_top=False, weights='imagenet')
        
        # Extract features
        features1 = vgg(tf.image.resize(images1 * 255, [224, 224]))
        features2 = vgg(tf.image.resize(images2 * 255, [224, 224]))
        
        # Compute perceptual distance
        lpips = tf.reduce_mean(tf.square(features1 - features2))
        
        return lpips.numpy()
    
    def diversity_score(self, generated_images):
        """Compute diversity score of generated samples"""
        
        batch_size = tf.shape(generated_images)[0]
        
        # Flatten images
        flattened = tf.reshape(generated_images, [batch_size, -1])
        
        # Compute pairwise distances
        distances = []
        for i in range(batch_size):
            for j in range(i + 1, batch_size):
                dist = tf.norm(flattened[i] - flattened[j])
                distances.append(dist)
        
        return tf.reduce_mean(distances).numpy()

# Run advanced analysis
print("=== Advanced Analysis ===")

# Analyze noise schedules
print("1. Analyzing noise schedules...")
analyze_noise_schedules()

# Analyze training dynamics (simplified)
print("\n2. Training dynamics analysis...")
if 'ddpm' in locals():
    sample_dataset = train_dataset.take(10)
    timestep_analysis = analyze_training_dynamics(ddmp, sample_dataset, num_batches=10)

# Quality metrics
print("\n3. Quality metrics...")
metrics = DiffusionMetrics()

if 'final_samples' in locals() and 'ddim_samples' in locals():
    # Convert samples to proper range [0, 1]
    final_samples_norm = tf.clip_by_value((final_samples + 1) / 2, 0, 1)
    ddim_samples_norm = tf.clip_by_value((ddim_samples + 1) / 2, 0, 1)
    real_samples_norm = (x_test[:8] + 1) / 2
    
    # Compute diversity
    diversity_ddpm = metrics.diversity_score(final_samples_norm)
    diversity_ddim = metrics.diversity_score(ddim_samples_norm)
    diversity_real = metrics.diversity_score(real_samples_norm)
    
    print(f"Diversity Scores:")
    print(f"  DDPM samples: {diversity_ddpm:.4f}")
    print(f"  DDIM samples: {diversity_ddim:.4f}")
    print(f"  Real samples: {diversity_real:.4f}")
    
    # Compute LPIPS between methods
    if final_samples_norm.shape[0] == ddim_samples_norm.shape[0]:
        lpips_comparison = metrics.compute_lpips_score(final_samples_norm, ddim_samples_norm)
        print(f"  LPIPS between DDPM and DDIM: {lpips_comparison:.4f}")

print("\nAdvanced analysis completed!")
```

## Summary

This comprehensive notebook demonstrated cutting-edge diffusion model implementations using tf.keras:

### Key Implementations

**1. Diffusion Fundamentals:**
- Noise scheduling with linear and cosine schedules  
- Forward diffusion process with noise addition
- U-Net architecture with time embeddings and attention
- Group normalization and residual connections

**2. DDPM (Denoising Diffusion Probabilistic Models):**
- Complete training pipeline with MSE noise prediction loss
- Reverse diffusion sampling with learned denoising steps  
- Proper parameterization and variance scheduling
- Training loop with loss tracking and visualization

**3. DDIM (Denoising Diffusion Implicit Models):**
- Deterministic sampling for faster generation
- Configurable number of inference steps
- ETA parameter for controlling stochasticity
- Significant speedup over DDPM sampling

**4. Advanced Techniques:**
- Classifier-free guidance for controllable generation
- Class-conditional generation with embedding layers
- Guidance scale for balancing quality vs diversity
- Fast sampling strategies and step scheduling

### Technical Achievements

- **Stable Training**: Proper noise scheduling prevents training instabilities
- **Fast Sampling**: DDIM enables 10-50x faster generation than DDPM  
- **Controllable Generation**: Classifier-free guidance enables class control
- **Quality Metrics**: FID, LPIPS, and diversity scores for evaluation

### Performance Insights

- **Cosine Schedule**: Better signal preservation than linear schedule
- **DDIM Sampling**: 20-50 steps vs 1000 for DDPM with similar quality
- **Guidance Scale**: Higher values (7.5+) improve quality but reduce diversity
- **U-Net Architecture**: Attention mechanisms crucial for high-quality results

### Applications Demonstrated

- Unconditional image generation
- Class-conditional synthesis  
- Fast sampling for interactive applications
- Quality assessment and comparison metrics

### Advanced Analysis

- Noise schedule comparison and optimization
- Training dynamics across timesteps
- Loss landscape analysis
- Comprehensive quality evaluation

### Next Steps

Continue to notebook 16 (TensorFlow Model Optimization) to learn model compression, quantization, and deployment optimization techniques that make these powerful generative models practical for production use.

Diffusion models represent the current state-of-the-art in generative modeling, offering superior quality and training stability compared to GANs while providing controllable generation capabilities.